# 📘 Reaction Kinetics: Rate Constant Determination> Fit first-order kinetic models to concentration data⏱️ **20-30 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Chemistry**---

## 🔬 Domain Background**Model:** $[A]_t = [A]_0 e^{-kt}$ (first-order)**Applications:** Reaction mechanism studies, catalyst evaluation---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef first_order(t, A0, k):    return A0 * jnp.exp(-k * t)

## Generate Data

In [ ]:
A0_true, k_true = 1.0, 0.05t = np.linspace(0, 100, 30)A_true = first_order(t, A0_true, k_true)A = A_true + np.random.normal(0, 0.02, len(t))plt.semilogy(t, A, 'o')plt.xlabel('Time (s)')plt.ylabel('[A] (M)')plt.tight_layout()
display(fig)
plt.close(fig)


## Fit Model

In [ ]:
popt, pcov = curve_fit(first_order, t, A, p0=[0.9, 0.04])A0_fit, k_fit = poptt_half = np.log(2) / k_fitprint(f'k: {k_fit:.4f} s⁻¹')print(f'Half-life: {t_half:.1f} s')

## Key Insights1. **Linear on ln[A] vs t** indicates first-order2. **Half-life constant** for first-order---